<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# ***A influência das opções no preço do tivo subjacente.***
---

$\rightarrow$ **esta célula deve ser removida na entrega do projeto**

# Objetivo

Neste trabalho você deve obrigatoriamente trabalhar em grupo de 2-4 alunos e desenvolver uma solução de aprendizado supervisionado para algum problema de interesse do grupo. Os problemas poderão ser de classificação ou de regressão.

## Descrição do Projeto

O trabalho deve ser original e desenvolvido em R ou Python notebook com um conjunto de dados próprios e originais definidos. Entendam como dados próprios e originais dados que são originais de empresas ou entidades, podendo ser bases públicas. Dados de ONGs, empresas privadas ou governo, são exemplos de bases válidas. Bases de repositórios públicos como Kaggle e UCI, que fornecem bases prontas não são originais. Tenha em mente: você deve construir uma base de dados nova para o seu problema, é essa a ideia de dados originais. Se tiver dúvidas consulte o professor antes de empregar uma base.

Os dados devem suportar um problema relevante e adequado para o aprendizado supervisionado. Assim, não podem ser empregadas bases artificiais e problemas muito simples. Devem ser evitadas também Séries Temporais como séries financeiras (como ações) e de dados climáticos que requerem outras técnicas não detalhadas aqui.

As bases devem conter um mínimo de 1000 instâncias. Consulte o professor para uso de um número menor de instâncias para dados próprios de uma empresa por exemplo.

Você deve implementar ao menos dois modelos, comparar e discutir os seus resultados. Seu notebook deve ser 100% executável e incluir todo necessário para construção dos dados, entendimento do problema e da solução.

Deve ainda ser entregue uma apresentação em **vídeo (até 5min)** dos resultados.



$\rightarrow$ **esta célula deve ser removida na entrega do projeto**

Siga este template para entrega do seu projeto. O código deste notebook precisa ser 100% executável, sem a necessidade de qualquer operação adicional como por exemplo a download, cópia ou alteração de arquivos. Crie assim repositórios públicos para suas bases de dados, por exemplo, disponibilizando-as no GitHub.

## Entregas na Tarefa do Moodle

1. Poste este Python notebook (.ipynb) modificado com o seu projeto.
2. Poste o endereço (link) do Python notebook do seu projeto compartilhado (público para leitura) nos comentários da tarefa
3. Um vídeo de até 5min com a apresentação do seu projeto em formato .mp4
4. Opcional, poste o .pptx da apresentação

## Vídeo

O vídeo deve contemplar:

1. Apresentação e Justificativa do seu problema
2. A abordagem dada ao problema (fonte dos dados, transformações, modelos avaliados)
3. A solução e seus resultados
4. Diferenciais e Melhorias Futuras do trabalho (modelos e técnicas diferenciadas que foram empregadas, limitações do modelo atual e como melhora-las etc.)







In [ ]:
#@title Identificação do Grupo

#@markdown Integrantes do Grupo (*informe \<TIA\>,\<nome\>*)
Aluno1 = '10423914, Humberto Guttau Bravo' #@param {type:"string"}
Aluno2 = '1043651, Ricardo Zulian de Souza Amaral' #@param {type:"string"}
Aluno3 = 'None' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}



# **Apresentação**

# Problema

O mercado dereivativos tem tido aumento constante de volume, muitas vezes induzindo fortes movimentações nos mercados dos seus ativos subjacentes, as ações.  
  
Nos extremos opostos do 'zero-sum game' que é o mercado de capitais estao os investidores de varejo e os formadores de meracdo. Estudando fatores como a proporção entre tomadores e lançadores e lançadpores cobertos/travados/descobertos é possível ententender se as posições abertas de derivativos são equilibradas ou polarizadas.  
  
Quando são muito polarizadas os formadores de mercado tendem a interferir no preço dos ativos subjacentes com o intuito de evitar (ou forçar) o exrcício das opções.

Buascamos aqui encontrar algoritmos de classificação que consigam identificar padrões nos contratos de opções que incitem a interferência do formador de mercado em um dia a partir das posições de derivativos consolidadas na véspera.  

O conhecimento antecipado desse pontos de interferência pode ser empregado como ferramenta para day-trade no ativo subjacente.  

# Referencial Teórico

*Apresente aqui o referencial teórico empregado. Empregue as referências empregadas que devem aparecer no final do trabalho. Lembre-se, existem  referências técnicas do domínio da solução, mas também referências do domínio do problema (por exemplo, se estiver classificando reclamações de clientes, existem inúmeras referências sobre o problema de negócio em si).*

# Metodologia

*Apresente e discuta a abordagem de solução empregada. Discuta os dados empregados, tratamento efetuados, variáveis preditoras e objetivo, técnicas empregadas.*



# Resultados

*Resuma aqui os resultados, principalmente incluindo tabelas e esquemas necessários.*

# **Implementação**

# Base de Dados

A base de dados tem origem nos data-dumps diários da B3. Existe um sistema próprio de um membro do grupo que faz o download diário dos arquivos texto gerados pela bolsa e os carrega como baixados em uma base de dados ms-sql.

A característica formal da base já garante consistência, como pode ser visto na ausência de nulos na base bruta. Nulos em campo específicos como volatilidade implicita são fruto de ausência de negócio no dia e não serão utilizados no estudo.




In [23]:
# seu código
import pandas as pd
import numpy as np
from sklearn.modelselection import train_test_split


rnd_seed = 42

# Carregando base de preços de fechamento de PETR4
df_p4 = pd.read_csv('petr4.csv.gz', compression='gzip')
df_p4_opc = pd.read_csv('petr4_opc.csv.gz', compression='gzip')
print("Base do Ativo:")
print(df_p4.info())
print(df_p4.sample(5))
print("\nBase do Derivativo:")
print(df_p4_opc.info())
print(df_p4_opc.sample(5))

print('\nBase do ativo entre: ', df_p4['tradedate'].min(), ' e ', df_p4['tradedate'].max())
print('\nBase do derivativo entre: ', df_p4_opc['tradedate'].min(), ' e ', df_p4_opc['tradedate'].max())

Base do Ativo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tradedate   643 non-null    object 
 1   ticker      643 non-null    object 
 2   price_last  643 non-null    float64
dtypes: float64(1), object(2)
memory usage: 15.2+ KB
None
      tradedate        ticker  price_last
544  2025-03-11  PETR4              34.10
436  2024-09-30  PETR4              36.01
316  2024-04-11  PETR4              39.30
266  2024-01-29  PETR4              40.57
218  2023-11-16  PETR4              35.55

Base do Derivativo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604847 entries, 0 to 604846
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   asset_id        604847 non-null  int64  
 1   tradedate       604847 non-null  object 
 2   ticker          604847 non-null  object 
 3  

**Sepração de PUTs/CALLS**  

Existem dois tipos de opções: opções de compra 'CALL' e opções de venda 'PUT'. os comportamentos de preço são invertidos e é necessário separa-las para processamento. Trbalharemos aqui apenas com opções de compra. A coluna com esta definição será eliminada.

In [24]:
print(len(df_p4_opc), 'registros totais.')
df_p4_opc = df_p4_opc[df_p4_opc['segment'] == 'EQUITY CALL']
df_p4_opc = df_p4_opc.drop(columns=['segment'])
print(len(df_p4_opc), 'registros de calls após a limpeza.')
print(df_p4_opc.sample(5))

604847 registros totais.
314083 registros de calls após a limpeza.
        asset_id   tradedate    ticker asset_ticker  strike style  \
419809     12440  2024-06-06  PETRB400        PETR4   37.12     E   
530552     13910  2024-09-30  PETRA389        PETR4   37.94     E   
217074      9558  2023-10-23   PETRA30        PETR4   24.54     E   
581832     14189  2024-12-02  PETRD469        PETR4   45.94     A   
418731     12434  2024-06-05  PETRH302        PETR4   17.20     A   

        pos_covered  pos_blocked  pos_uncovered      oi  lenders  borrowers  \
419809         3200        55400          32100   90700       20         25   
530552          500         5900          18300   24700       12          5   
217074         7600       177500          42900  228000       31         23   
581832            0        18400          81800  100200        2          2   
418731         5000       146000           7800  158800       25         16   

              iq  timo_to_expiry  moneyness

O rótulo de classificação será definido como se cada contrato se comportou como uma 'barreira' no pregão seguinte: o preço do ativo não conseguiu romper o strike da opção. Para isso precisamos deslocar os preços de fechamento do ativo para incorporação na base de derivativos.

In [25]:
#Criando a coluna 'next_price_last' contendo o fechamento do dia seguinte do ativo.

df_p4['next_price_last'] = df_p4['price_last'].shift(-1).fillna(0)
print(df_p4.head())

    tradedate        ticker  price_last  next_price_last
0  2023-01-02  PETR4              22.92            22.34
1  2023-01-03  PETR4              22.34            23.05
2  2023-01-04  PETR4              23.05            23.88
3  2023-01-05  PETR4              23.88            23.74
4  2023-01-06  PETR4              23.74            23.87


In [26]:
#incorporando a coluna next_price_last no dataframe de opções
df_p4_opc = pd.merge(df_p4_opc, df_p4[['tradedate', 'next_price_last']], on='tradedate', how='left')
print(df_p4_opc.head())

   asset_id   tradedate    ticker asset_ticker  strike style  pos_covered  \
0      1231  2023-01-02    PETRA1        PETR4   12.51     A        29700   
1      1231  2023-01-02  PETRA108        PETR4   13.76     E         1900   
2      1231  2023-01-02  PETRA130        PETR4   13.01     A         3600   
3      1231  2023-01-02  PETRA132        PETR4   13.26     E        34400   
4      1231  2023-01-02  PETRA140        PETR4    1.39     A            0   

   pos_blocked  pos_uncovered      oi  lenders  borrowers        iq  \
0        10000         127100  166800       15         28  0.535714   
1        26900          11200   40000        4         11  0.363636   
2        18100          13600   35300        3          7  0.428571   
3        26300         135100  195800        6         26  0.230769   
4        60003           1251   61254       14          5  2.800000   

   timo_to_expiry  moneyness  vol_implied  next_price_last  
0        0.055556  -0.454188     1.531464        

# Revisão dos campos para preparação dos dados  
**asset_id** índice do ativo relacionado em outra tabela *remover*  
**tradedate** data de negociação, não vai ser utlizado no modelo mas é interessante para visiualização *manter*     
**ticker** código de negociação do contrato não vai ser utlizado no modelo mas é interessante para visiualização *manter*      
**asset_ticker**  código do ativo subjacente, igual em todas *remover*  
**strike** preço do ativo para exercício da opção, utilizado na rotulagem *manter*  
**style** tipo de opção, americana ou européia, tem influência em negociação *manter*  
**pos_covered** tamanho da posição vendida (número de contratos abertos) coberta *manter*  
**pos_blocked** tamanho da posição vendida (número de contratos abertos) travada *manter*  
**pos_uncovered** tamanho da posição vendida (número de contratos abertos) descoberta *manter*  
**oi** tamanho da posição (número de contratos abertos) *manter*  
**lenders** titulares de opção *manter*   
**borrowers** lançadores de opção *manter*  
**iq** titulares/lançadores *manter* https://www.clubedopairico.com.br/distorcoes-no-indice-de-qualidade-das-opcoes-o-que-isso-significa/39249  
**timo_to_expiry** prazo até o exercíco (em anos base 360dd) *manter*  
**moneyness** dentro do dinheiro / no dinheiro / fora do dinheiro. Distância relativa entre o preço do ativo e o strike da opção *manter*  
**vol_implied** volatilidade implicita calculada no dia *remover*   
**next_price_last** preço de fechamento do ativo subjecente no pregão de negociação seguinte *manter*  

In [27]:
df_p4_opc = df_p4_opc.drop(columns=['asset_id', 'asset_ticker', 'vol_implied'])


In [28]:
df_p4_opc.head()

,tradedate,ticker,strike,style,pos_covered,pos_blocked,pos_uncovered,oi,lenders,borrowers,iq,timo_to_expiry,moneyness,next_price_last
0,2023-01-02,PETRA1,12.51,A,29700,10000,127100,166800,15,28,0.535714,0.055556,-0.454188,22.34
1,2023-01-02,PETRA108,13.76,E,1900,26900,11200,40000,4,11,0.363636,0.055556,-0.399651,22.34
2,2023-01-02,PETRA130,13.01,A,3600,18100,13600,35300,3,7,0.428571,0.055556,-0.432373,22.34
3,2023-01-02,PETRA132,13.26,E,34400,26300,135100,195800,6,26,0.230769,0.055556,-0.421466,22.34
4,2023-01-02,PETRA140,1.39,A,0,60003,1251,61254,14,5,2.800000,0.055556,-0.939354,22.34


## **Modelo 1**

*Descreva resumidamente o modelo empregado.*


### **Modelo 1:** Preparação dos Dados
Preparação básica com conversão de valores alfanuméricos, remoção de multicolinearidade e criação do rótulo

In [38]:
# o campo 'style' será converito para um binário 'american' com valor 1 para americans e 0 para européias
df_opc_modelo1 = df_p4_opc.copy()
df_opc_modelo1['american'] = np.where(df_opc_modelo1['style'] == 'A', 1, 0)
df_opc_modelo1 = df_opc_modelo1.drop(columns=['style']) # removendo coluna 'style'

# criação do rótulo 'barrrier' baseado no comportamento do preço do ativo subjacente no dia seguinte
df_opc_modelo1['barrier'] = np.where(df_opc_modelo1['next_price_last'] > df_opc_modelo1['strike'] , 1, 0)
df_opc_modelo1 = df_opc_modelo1.drop(columns=['next_price_last']) # removendo coluna 'next_price_last', não será utilizada novamente
#strike foi mantido pois pode ser útil para visualização


# oi = pos_covered pos_blocked pos_uncovered. Manter oi como número de contratos, converter pos_covered e pos_uncovered para rsão sobre oi 
# e eliminar pos_blocked que pode ser dericado das outras três variáveis
df_opc_modelo1['covered_ratio'] = df_opc_modelo1['pos_covered']/ df_opc_modelo1['oi']
df_opc_modelo1['uncovered_ratio'] = df_opc_modelo1['pos_uncovered']/ df_opc_modelo1['oi']
df_opc_modelo1 = df_opc_modelo1.drop(columns=['pos_covered', 'pos_blocked',  'pos_uncovered']) # descartando colunas processadas

#iq = lenders/borrowers. Manter iq e lenders, descartar borrowers. A razão é importante e o número total de titulares também,
#indica quão popular é uma posição. O número de lançadores vai ser removido pois pode ser derivado dos outros dois.
df_opc_modelo1 = df_opc_modelo1.drop(columns=['borrowers'])


df_opc_modelo1.sample(5)

,tradedate,ticker,strike,oi,lenders,iq,timo_to_expiry,moneyness,american,barrier,covered_ratio,uncovered_ratio
10287,2023-01-26,PETRC315,31.51,51400,5,1.666667,0.134921,0.202672,1,0,0.000000,1.000000
119754,2023-11-08,PETRL447,35.79,3711200,149,1.034722,0.103175,0.047410,1,0,0.026784,0.088004
156760,2024-02-07,PETRB216,21.26,369600,6,0.250000,0.019841,-0.496209,1,1,0.772186,0.184524
105111,2023-10-05,PETRD303,10.79,2198600,45,7.500000,0.527778,-0.670333,1,1,0.226963,0.683117
9218,2023-01-24,PETRB560,52.76,2306600,25,3.125000,0.071429,0.983459,0,0,0.015477,0.007067


In [50]:
#divisão da base em train & test
train_m1, test_m1 = train_test_split(df_opc_modelo1,test_size=0.3,random_state = rnd_seed)
print('Treinamento com ', len(train_m1), ' registros') 
print('Teste com ', len(test_m1), ' registros') 
X_train_m1 = train_m1.drop(columns=['barrier','tradedate','ticker','strike']) 
y_train_m1 = train_m1['barrier']
X_test_m1 = test_m1.drop(columns=['barrier','tradedate','ticker','strike']) 
y_test_m1 = test_m1['barrier']
X_train_m1.head()

Treinamento com  219858  registros
Teste com  94225  registros


,oi,lenders,iq,timo_to_expiry,moneyness,american,covered_ratio,uncovered_ratio
229283,15600,2,0.250000,0.238095,-0.588173,0,0.173077,0.826923
104940,13800,5,1.250000,0.194444,0.181484,0,0.072464,0.927536
87345,349800,14,7.000000,0.642857,0.632184,1,0.000000,0.000000
199364,56400,8,0.421053,0.214286,-0.006769,0,0.010638,0.226950
160290,54000,1,0.333333,0.063492,0.063830,1,0.000000,0.000000


### **Modelo 1:** Modelo


In [ ]:
# seu código

### **Modelo 1:** Resultados


In [ ]:
# seu código

## **Modelo 2**

*Descreva resumidamente o modelo empregado.*


## **Modelo 2:** Preparação dos Dados
A mesma preparação de básica de dados será feita. Passos adicionais seram incluídos para a normalização de valores com escalas disconexas e negativos para emprego em algoritmos sensíveis.


In [56]:
# o campo 'style' será converito para um binário 'american' com valor 1 para americans e 0 para européias
df_opc_modelo2 = df_p4_opc.copy()
df_opc_modelo2['american'] = np.where(df_opc_modelo2['style'] == 'A', 1, 0)
df_opc_modelo2 = df_opc_modelo2.drop(columns=['style']) # removendo coluna 'style'

# criação do rótulo 'barrrier' baseado no comportamento do preço do ativo subjacente no dia seguinte
df_opc_modelo2['barrier'] = np.where(df_opc_modelo2['next_price_last'] > df_opc_modelo2['strike'] , 1, 0)
df_opc_modelo2 = df_opc_modelo2.drop(columns=['next_price_last']) # removendo coluna 'next_price_last', não será utilizada novamente
#strike foi mantido pois pode ser útil para visualização


# oi = pos_covered pos_blocked pos_uncovered. Manter oi como número de contratos, converter pos_covered e pos_uncovered para rsão sobre oi 
# e eliminar pos_blocked que pode ser dericado das outras três variáveis
df_opc_modelo2['covered_ratio'] = df_opc_modelo2['pos_covered']/ df_opc_modelo2['oi']
df_opc_modelo2['uncovered_ratio'] = df_opc_modelo2['pos_uncovered']/ df_opc_modelo2['oi']
df_opc_modelo2 = df_opc_modelo2.drop(columns=['pos_covered', 'pos_blocked',  'pos_uncovered']) # descartando colunas processadas

#iq = lenders/borrowers. Manter iq e lenders, descartar borrowers. A razão é importante e o número total de titulares também,
#indica quão popular é uma posição. O número de lançadores vai ser removido pois pode ser derivado dos outros dois.
df_opc_modelo2 = df_opc_modelo2.drop(columns=['borrowers'])

df_opc_modelo2.sample(5)

,tradedate,ticker,strike,oi,lenders,iq,timo_to_expiry,moneyness,american,barrier,covered_ratio,uncovered_ratio
25275,2023-03-09,PETRC371,25.76,1995800,101,0.367273,0.023810,0.017780,0,0,0.394529,0.405802
294435,2024-11-13,PETRB360,31.18,7600,9,3.000000,1.257937,-0.154555,0,1,0.000000,0.513158
6186,2023-01-16,PETRF403,23.80,228700,8,2.666667,0.408730,-0.009983,1,1,0.002186,0.997814
20051,2023-02-23,PETRE29,29.51,280500,4,0.400000,0.230159,0.111488,1,0,0.970053,0.029947
47054,2023-05-11,PETRF373,17.97,437000,10,0.357143,0.099206,-0.293354,1,1,0.030206,0.165217


In [57]:
#divisão da base em train & test
train_m2, test_m2 = train_test_split(df_opc_modelo1,test_size=0.3,random_state = rnd_seed)
print('Treinamento com ', len(train_m2), ' registros') 
print('Teste com ', len(test_m2), ' registros') 
X_train_m2 = train_m2.drop(columns=['barrier','tradedate','ticker','strike']) 
y_train_m2 = train_m2['barrier']
X_test_m2 = test_m2.drop(columns=['barrier','tradedate','ticker','strike']) 
y_test_m2 = test_m2['barrier']

#normalização dos valores
from sklearn.preprocessing import MinMaxScaler  # Ou StandardScaler - minmax aqui para não ter negativos

print(X_train_m2.head())

# Create and fit the scaler
scaler_m2 = MinMaxScaler()

X_train_m2 = pd.DataFrame(scaler_m2.fit_transform(X_train_m2), index=X_train_m2.index, columns=X_train_m2.columns)
print('Normalizado: ')
print(X_train_m2.head())

Treinamento com  219858  registros
Teste com  94225  registros
            oi  lenders        iq  timo_to_expiry  moneyness  american  \
229283   15600        2  0.250000        0.238095  -0.588173         0   
104940   13800        5  1.250000        0.194444   0.181484         0   
87345   349800       14  7.000000        0.642857   0.632184         1   
199364   56400        8  0.421053        0.214286  -0.006769         0   
160290   54000        1  0.333333        0.063492   0.063830         1   

        covered_ratio  uncovered_ratio  
229283       0.173077         0.826923  
104940       0.072464         0.927536  
87345        0.000000         0.000000  
199364       0.010638         0.226950  
160290       0.000000         0.000000  
Normalizado: 
              oi   lenders        iq  timo_to_expiry  moneyness  american  \
229283  0.000484  0.000697  0.005235        0.112811   0.075019       0.0   
104940  0.000428  0.002787  0.027465        0.091778   0.215353       0.0   
8

### **Modelo 2:** Modelo


In [ ]:
# seu código

### **Modelo 2:** Resultados


In [ ]:
# seu código

# **Conclusão**

*Apresente a conclusão do seu estudo comparando ainda os resultados obtidos com o referencial teórico apresentado.*



# **Referências**

*Indique as referências empregadas, incluindo as fontes de dados.*

---

In [ ]:
#@title Avaliação
Completo = 10 #@param {type:"slider", min:0, max:10, step:1}
#@markdown Projeto cumpre todos os itens pedidos.
Relevancia = 9 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As seleções de dados e eventos para análise são relevantes e justificados.
Tecnicas = 6 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As técnicas de empregadas são adequadas e corretamente aplicadas.
Apresentacao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown A apresentação dos resultados é clara e objetiva.
Analise = 8 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As premissas de análise se justificam e a analise é correta.
Conclusao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As conclusões são justificadas e relevantes
Bonus = 0.5 #@param {type:"slider", min:0, max:1, step:0.5}
#@markdown A critério do professor por inovações na abordagem e no uso de técnicas de Análise de Dados








In [ ]:
#@markdown ### Nota Final
nota = Completo + Relevancia + Tecnicas + Apresentacao + Analise + Conclusao

nota = nota / 6 + Bonus

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
alunos

Nota final do trabalho 8.3



,tia,nome,nota
0,1115665,ADRIANA FUJITA,8.3
1,1115677,DANIEL HENRIQUE,8.3
